In [1]:
# !pip install openpyxl

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv("/data/Augmented_data.csv")
df["Surname"] = df["Surname"].progress_apply(lambda x: str(x).capitalize())
df.head()

100%|██████████| 293712/293712 [00:00<00:00, 975618.53it/s]


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,date
0,15773098,Ch'in,850.000000,Spain,Male,55,26,0.000000,2,0,0,0.0,0,0,5,GOLD,1699.0,1-5-2022
1,15609340,Burks,850.000000,France,Male,40,15,377456.690406,1,1,1,1.0,0,0,3,GOLD,756.0,1-1-2022
2,15639004,Chiemezie,789.582101,France,Male,72,2,0.000000,2,1,1,1.0,1,0,5,DIAMOND,833.0,1-8-2020
3,15956058,Dotson,850.000000,France,Male,46,23,359827.773635,1,1,0,1.0,0,0,3,GOLD,1996.0,1-2-2022
4,15735950,Winslow,850.000000,Germany,Male,106,32,101424.101476,2,0,1,0.0,0,0,5,DIAMOND,2476.0,1-9-2022


In [3]:
cus_st = {}
for cus, st in zip(df["CustomerId"],df["Geography"]):
    if cus not in cus_st:
        cus_st[cus] = [st]
    else:
        if st not in cus_st[cus]:
            cus_st[cus].append(st)

In [4]:
df["Geography"] =  df["CustomerId"].progress_apply(lambda x: cus_st[x][0])

100%|██████████| 293712/293712 [00:00<00:00, 979661.45it/s]


In [5]:
cities = pd.read_excel("US_cities.xlsx")
state_city = {}
for st, ct in zip(cities["state"], cities["city"]):
    if st not in state_city:
        state_city[st] = [ct]
    else:
        state_city[st].append(ct)
        
        
state_mapper = {}
for st in df["Geography"]:
    if st not in state_mapper:
        state_mapper[st] = np.random.choice(list(state_city.keys()))
        
city_mapper = {}
for cust_id, st in zip(df["CustomerId"],df["Geography"]):
    if cust_id not in city_mapper:
        city_mapper[cust_id] = np.random.choice(state_city[state_mapper[st]])

In [6]:
df["City"] = df["CustomerId"].progress_apply(lambda x: city_mapper[x])
df["month"] = df["date"].progress_apply(lambda x: x.split("-")[1])
df["year"] = df["date"].progress_apply(lambda x: x.split("-")[-1])
df["CreditScore"] = df["CreditScore"].progress_apply(lambda x: round(x,0))
df["Account_type"] = df["Tenure"].progress_apply(lambda x: "Gold" if x > 15 else "Premium" if x <= 15 and x >=8 else "Normal")

100%|██████████| 293712/293712 [00:00<00:00, 1225245.95it/s]


In [7]:
df["Other Accounts"] = df["Tenure"].progress_apply(lambda x: 1 if x > 7 else 0)

100%|██████████| 293712/293712 [00:00<00:00, 1057393.55it/s]


In [8]:
df["Salary Account"] = df["Satisfaction Score"].progress_apply(lambda x: 1 if x >= 3 else 0 )

100%|██████████| 293712/293712 [00:00<00:00, 1052676.14it/s]


In [9]:
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,...,Satisfaction Score,Card Type,Point Earned,date,City,month,year,Account_type,Other Accounts,Salary Account
0,15773098,Ch'in,850.0,Spain,Male,55,26,0.000000,2,0,...,5,GOLD,1699.0,1-5-2022,Rapid City,5,2022,Gold,1,1
1,15609340,Burks,850.0,France,Male,40,15,377456.690406,1,1,...,3,GOLD,756.0,1-1-2022,Portland,1,2022,Premium,1,1
2,15639004,Chiemezie,790.0,France,Male,72,2,0.000000,2,1,...,5,DIAMOND,833.0,1-8-2020,Portland,8,2020,Normal,0,1
3,15956058,Dotson,850.0,France,Male,46,23,359827.773635,1,1,...,3,GOLD,1996.0,1-2-2022,Portland,2,2022,Gold,1,1
4,15735950,Winslow,850.0,Germany,Male,106,32,101424.101476,2,0,...,5,DIAMOND,2476.0,1-9-2022,Fayetteville,9,2022,Gold,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293707,15770424,Onyeorulu,678.0,Germany,Male,40,7,128360.739022,2,1,...,1,PLATINUM,1208.0,1-8-2020,Fort Smith,8,2020,Normal,0,0
293708,15626507,Chukwubuikem,725.0,France,Male,27,1,196831.949799,1,1,...,1,GOLD,1041.0,1-8-2020,Portland,8,2020,Normal,0,0
293709,15743478,Johnson,734.0,Germany,Male,39,8,60297.705608,2,1,...,1,SILVER,669.0,1-3-2020,Conway,3,2020,Premium,1,0
293710,15087982,Armendariz,850.0,France,Male,81,54,553571.976162,1,0,...,2,PLATINUM,1818.0,1-6-2023,Portland,6,2023,Gold,1,0


In [10]:
df.describe()

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,Other Accounts,Salary Account
count,2.937120e+05,293712.000000,293712.000000,293712.000000,2.937120e+05,293712.000000,293712.000000,293712.000000,293712.000000,293712.000000,293712.000000,293712.000000,293712.000000,293712.000000,293712.000000
mean,1.554317e+07,808.427810,54.059766,21.680936,2.323903e+05,1.538633,0.711895,0.547492,3408.456024,0.140628,0.007405,3.025675,1367.832390,0.645891,0.610081
std,2.680517e+05,78.699068,26.923795,25.161427,3.253166e+05,0.512203,0.452881,0.497740,21025.361942,0.347638,0.085734,1.406359,1002.442674,0.478243,0.487733
min,1.500005e+07,350.000000,18.000000,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,119.000000,0.000000,0.000000
25%,1.532483e+07,808.000000,36.000000,6.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,712.000000,0.000000,0.000000
50%,1.560255e+07,850.000000,44.000000,10.000000,1.463362e+05,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,3.000000,1070.000000,1.000000,1.000000
75%,1.575057e+07,850.000000,64.000000,29.000000,3.011300e+05,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.000000,1671.000000,1.000000,1.000000
max,1.600000e+07,850.000000,187.000000,116.000000,3.082030e+06,4.000000,1.000000,1.000000,199992.480000,1.000000,1.000000,5.000000,8403.000000,1.000000,1.000000


In [204]:
df.to_csv("/data/bank_customer_churn_data.csv", index=False)